In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor


In [ ]:
data = pd.read_csv('../input/train.csv', parse_dates=["release_date"])
test_data = pd.read_csv('../input/test.csv', parse_dates=["release_date"])
data.head()

In [ ]:
def proc_json(string, key):
    try:
        data = eval(string)
        return ",".join([d[key] for d in data])
    except:
        return ''

def proc_json_len(string):
    try:
        data = eval(string)
        return len(data)
    except:
        return 0

data.genres = data.genres.apply(lambda x: proc_json(x, 'name'))
test_data.genres = test_data.genres.apply(lambda x: proc_json(x, 'name'))

data.production_companies = data.production_companies.apply(lambda x: proc_json(x, 'name'))
test_data.production_companies = test_data.production_companies.apply(lambda x: proc_json(x, 'name'))

data.production_countries = data.production_countries.apply(lambda x: proc_json(x, 'iso_3166_1'))
test_data.production_countries = test_data.production_countries.apply(lambda x: proc_json(x, 'iso_3166_1'))

data.spoken_languages = data.spoken_languages.apply(lambda x: proc_json(x, 'iso_639_1'))
test_data.spoken_languages = test_data.spoken_languages.apply(lambda x: proc_json(x, 'iso_639_1'))

data.Keywords = data.Keywords.apply(lambda x: proc_json(x, 'name'))
test_data.Keywords = test_data.Keywords.apply(lambda x: proc_json(x, 'name'))

data.cast = data.cast.apply(proc_json_len)
test_data.cast = test_data.cast.apply(proc_json_len)

data.crew = data.crew.apply(proc_json_len)
test_data.crew = test_data.crew.apply(proc_json_len)

print('done')

In [ ]:
genres = []
for idx, val in data.genres.iteritems():
    gen_list = val.split(',')
    for gen in gen_list:
        if gen == '':
            continue

        if gen not in genres:
            genres.append(gen)
            

genre_column_names = []
for gen in genres:
    col_name = 'genre_' + gen.replace(' ', '_')
    data[col_name] = data.genres.str.contains(gen).astype('uint8')
    test_data[col_name] = test_data.genres.str.contains(gen).astype('uint8')
    genre_column_names.append(col_name)
    
langs = []
for idx, val in data.original_language.iteritems():
    lang_list = val.split(',')
    for lang in lang_list:
        if lang == '':
            continue

        if lang not in langs:
            langs.append(lang)
            

langs_column_names = []
for lang in langs:
    col_name = 'lang_' + lang.replace(' ', '_')
    data[col_name] = data.original_language.str.contains(lang).astype('uint8')
    test_data[col_name] = test_data.original_language.str.contains(lang).astype('uint8')
    langs_column_names.append(col_name)

In [ ]:
data['year'] = data['release_date'].dt.year
#data['day'] = data['release_date'].dt.day
data['month'] = data['release_date'].dt.month
data['release_dayofweek'] = data['release_date'].dt.dayofweek
data['release_quarter'] = data['release_date'].dt.quarter
data['belongs_to_collection'] = data['belongs_to_collection'].apply(lambda x: isinstance(x, str))
data['tagline'] = data['tagline'].apply(lambda x: isinstance(x, str))
data['homepage'] = data['homepage'].apply(lambda x: isinstance(x, str))
data['runtime'].fillna(data['runtime'].mean(), inplace=True)
#data['revenue'] = np.log1p(data['revenue'])
#data['budget'] = np.log1p(data['budget'])
data.loc[data['year'] > 2018, 'year'] = data.loc[data['year'] > 2018, 'year'].apply(lambda x: x - 100)
data['_budget_year_ratio'] = data['budget']/(data['year']*data['year'])


test_data['year'] = test_data['release_date'].dt.year
test_data['month'] = test_data['release_date'].dt.month
test_data['release_dayofweek'] = test_data['release_date'].dt.dayofweek
test_data['release_quarter'] = test_data['release_date'].dt.quarter
test_data['year'].fillna(test_data['year'].mean(), inplace=True)
test_data['month'].fillna(test_data['month'].mean(), inplace=True)
test_data['release_dayofweek'].fillna(test_data['release_dayofweek'].mean(), inplace=True)
test_data['release_quarter'].fillna(test_data['release_quarter'].mean(), inplace=True)
test_data['belongs_to_collection'] = test_data['belongs_to_collection'].apply(lambda x: isinstance(x, str))
test_data['tagline'] = test_data['tagline'].apply(lambda x: isinstance(x, str))
test_data['homepage'] = test_data['homepage'].apply(lambda x: isinstance(x, str))
test_data['runtime'].fillna(test_data['runtime'].mean(), inplace=True)
#test_data['budget'] = np.log1p(test_data['budget'])
test_data.loc[test_data['year'] > 2018, 'year'] = test_data.loc[test_data['year'] > 2018, 'year'].apply(lambda x: x - 100)
test_data['_budget_year_ratio'] = test_data['budget']/(test_data['year']*test_data['year'])


y = data.revenue
X =            data[['popularity',
                   #  'budget',
                     'runtime','belongs_to_collection','homepage','year','tagline','cast','crew',
                   # 'genre_Adventure', 'genre_Fantasy', 'genre_Science_Fiction', 'genre_Action', 'genre_Family', 'genre_Thriller',
                    # 'lang_en', 'lang_zh',  
                     'release_dayofweek','release_quarter', 'month','_budget_year_ratio']]
#X = data.drop(['genres', 'imdb_id', 'original_language', 'original_title', 'overview', 
#               'poster_path', 'production_companies', 'production_countries', 'release_date', 
#               'spoken_languages', 'status', 'title', 'Keywords', 'revenue' ,'day'], axis=1)

X_check = test_data[['popularity',
                   #  'budget',
                     'runtime','belongs_to_collection','homepage','year','tagline','cast','crew',
                    #'genre_Adventure', 'genre_Fantasy', 'genre_Science_Fiction', 'genre_Action', 'genre_Family', 'genre_Thriller',
                    # 'lang_en', 'lang_zh',  
                     'release_dayofweek','release_quarter', 'month','_budget_year_ratio']]
#X_check = test_data.drop(['genres', 'imdb_id', 'original_language', 'original_title', 'overview', 
#               'poster_path', 'production_companies', 'production_countries', 'release_date', 
#               'spoken_languages', 'status', 'title', 'Keywords'], axis=1)

#data.head()
X.head()

In [ ]:
import seaborn as sns
#data['isTaglineNA'] = 0
#data.loc[pd.isnull(train['tagline']) ,"isTaglineNA"] = 1
sns.catplot(x="runtime", y="revenue", data=data)
#plt.title('collection');

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20,20))         # Sample figsize in inches
#sns.heatmap(df1.iloc[:, 1:6:], annot=True, linewidths=.5, ax=ax)

data.corr(method ='pearson') 
#import seaborn as sns

#Var_Corr = data.corr()
#plot the heatmap and annotation on it
#sns.heatmap(Var_Corr, xticklabels=Var_Corr.columns, yticklabels=Var_Corr.columns, annot=True, ax=ax)
#print(data[data.columns[1:]].corr()['revenue'])

#correl=data[data.columns[1:]].corr()['revenue']
#correl2 = correl.sort_values()
#print(correl2)
#correl.sort_values(by=correl[1], axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
#correl=data.year
#correl2 = correl.sort_values()
#print(correl2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from math import sqrt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#xgb_model = XGBRegressor(n_estimators=1000)
#xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)
#xgb_val_predictions = xgb_model.predict(X_test)

MyRegressor = RandomForestRegressor(
    max_features=3, 
    n_estimators=100, 
    random_state=42)
MyRegressor.fit(X_train, y_train)
predictions = MyRegressor.predict(X_test)



score = mean_squared_log_error(y_test, predictions)
#score2 = mean_absolute_error(y_test, xgb_val_predictions)
print(sqrt(score))
#62448018.91106667 - popularity
#47313795.4944 - + budget
#44651654.391333334 - + website_present
#43427197.428799994 - +language one-hot-encoded
#43183219.09346666 - +runtime 0/mean, -lang
#42606840.899333335 - collection_present
#40628122.7004 - +year #38786032.13675 - same XGB
#37955117.4664443 - 10depth

In [ ]:
#MyRegressor.fit(X, y)
#xgb_model_on_full_data = XGBRegressor(n_estimators=1000)
# fit rf_model_on_full_data on all data from the training data
#xgb_model_on_full_data.fit(X, y, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)

In [ ]:
new_predictions = MyRegressor.predict(X_check)
#new_predictions = xgb_model_on_full_data.predict(X_check)

sub = pd.read_csv('../input/sample_submission.csv')
sub['revenue'] = new_predictions
sub.to_csv("submission.csv", index=False)

print('done')

In [ ]:
#len(test_data)
test_data.isnull().any()
#X_check.isnull().any()